In [55]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time
import csv
import re

driver = webdriver.Chrome('/Users/Patrick/Downloads/chromedriver')
driver.get('https://www.edx.org/course?language=English')

csv_file = open('courses_whole.csv', 'w')

writer = csv.writer(csv_file)
writer.writerow(['course_link','price', 'title', 
                 'subject', 'level', 'institution', 'length', 
                 'prerequisites', 'short_description', 'effort'])

wait = WebDriverWait(driver, 2)


In [4]:
###this code scrapes the main course list page, returning a list###
###of all english course links###

num_classes_str = driver.find_element_by_xpath('//span[@class="js-count result-count"]').text		#get the total number of courses
num_classes = int((re.findall(r'\d+', num_classes_str))[0])											#convert course number to int

page = 0

while page < num_classes:
	driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")

	try:
		WebDriverWait(driver, 10).until(EC.visibility_of_element_located((By.XPATH, '//div[@class="loading"]')))
		page += 1
	except Exception as e:
		print(e)
		print(page)
		break


courses = driver.find_elements_by_xpath('//div[@class="discovery-card-inner-wrapper"]/a[@class="course-link"]')
print('='*50)
print(len(courses))
print(courses[0])
print(courses[1])
print(courses[2])
print(courses[3])
print('='*50)
course_links = []
for course in courses:
	course_links.append(course.get_attribute('href'))

print('='*50)
print(len(course_links))
print(course_links[0])


Message: 

1257
1257
<selenium.webdriver.remote.webelement.WebElement (session="aae2888fc5e42e2618d93dbacef69838", element="0.5939802481140002-3")>
<selenium.webdriver.remote.webelement.WebElement (session="aae2888fc5e42e2618d93dbacef69838", element="0.5939802481140002-4")>
<selenium.webdriver.remote.webelement.WebElement (session="aae2888fc5e42e2618d93dbacef69838", element="0.5939802481140002-5")>
<selenium.webdriver.remote.webelement.WebElement (session="aae2888fc5e42e2618d93dbacef69838", element="0.5939802481140002-6")>
1257
https://www.edx.org/course/introduction-web-accessibility-microsoft-dev240x


In [27]:
###testing course info scraping with three links
course_links_test1 = ['https://www.edx.org/course/apr-italian-language-culture-wellesleyx-apita-x', 
                     'https://www.edx.org/course/ramp-ap-biology-weston-high-school-bio101x-1',
                     'https://www.edx.org/course/selling-ideas-how-influence-others-get-wharton-sellingideas101x-2']

In [53]:
###testing course info scraping with 50 links
course_links_test3 = course_links[:50]


In [56]:
def get_title():
    try:
        title = driver.find_element_by_xpath('.//*[@id="course-info-page"]//h1[@id="course-intro-heading"]').text
    except:
        title = 'Missing'
    finally:
        return title

def get_short_description():
    try:
        short_description = driver.find_element_by_xpath('.//*[@id="course-info-page"]//p[@class="course-intro-lead-in"]').text
    except:
        short_description = 'Missing'
    finally:
        return short_description

def get_length():
    try:
        length = driver.find_element_by_xpath('.//*[@id="course-summary-area"]//li[@data-field="length"]/span[2]').text
    except:
        length = 'Missing'   
    finally:
        return length

def get_effort():
    try:
        effort = driver.find_element_by_xpath('.//*[@id="course-summary-area"]//li[@data-field="effort"]//span[@class="block-list__desc"]').text
    except:
        effort = 'Missing'
    finally:
        return effort

def get_price():
    try:
        price = driver.find_element_by_xpath('.//*[@id="course-summary-area"]//li[@data-field="price"]//span[@class="block-list__desc"]/span[@class="uppercase"]').text
    except:
        try:                               
            price = driver.find_element_by_xpath('.//*[@id="course-summary-area"]//span[@class="fa fa-tag fa-lg"]]/../parent::span/following-sibling::span').text()
        except:
            price = "Missing"
    finally:
        return price
    
def get_institution():
    try:
        institution = driver.find_element_by_xpath('.//*[@id="course-summary-area"]//li[@data-field="school"]/span[2]/a').text
    except:
        institution = 'Missing'
    finally:
        return institution

def get_subject():
    try:
        subject = driver.find_element_by_xpath('.//*[@id="course-summary-area"]//li[@data-field="subject"]/span[2]/a').text
    except:
        subject = 'Missing'
    finally:
        return subject
    
def get_level():
    try:
        level = driver.find_element_by_xpath('.//*[@id="course-summary-area"]//li[@data-field="level"]//span[@class="block-list__desc"]').text
    except:
        level = 'Missing'
    finally:
        return level

def get_prerequisites():
    try:
        prerequisites = driver.find_element_by_xpath('.//*[@id="course-summary-area"]/div[2]/p').text
    except:
        try:
            prerequisites = driver.find_element_by_xpath('.//*[@id="course-summary-area"]/div[2]/ul/li[1]')
        except:
            prerequisites = 'Missing'
    finally:
        return prerequisites

    
for course_link in course_links:
    course_dict = {}
    driver = webdriver.Chrome('/Users/Patrick/Downloads/chromedriver')
    driver.get(course_link)
    
        
    course_dict['link'] = course_link
    course_dict['title'] = get_title()
    course_dict['short_description'] = get_short_description()
    course_dict['length'] = get_length()
    course_dict['effort'] = get_effort()
    course_dict['price'] = get_price()
    course_dict['institution'] = get_institution()
    course_dict['subject'] = get_subject()
    course_dict['level'] = get_level()
    course_dict['prerequisites'] = get_prerequisites()
    writer.writerow(course_dict.values())
    driver.close()
    
csv_file.close()

In [13]:

            
	try:
		title = driver.find_element_by_xpath('.//*[@id="course-info-page"]/header/div/div/div[2]/h1[@id="course-intro-heading"]').text

		short_description = driver.find_element_by_xpath('.//*[@id="course-info-page"]/header/div/div/div[2]/p[@class="course-intro-lead-in"]').text

		length = driver.find_element_by_xpath('.//*[@id="course-summary-area"]/ul/li[1]/span[2][@class="block-list__desc"]').text

		effort = driver.find_element_by_xpath('.//*[@id="course-summary-area"]/ul/li[2]/span[2][@class="block-list__desc"]').text

		price = driver.find_element_by_xpath('.//*[@id="course-summary-area"]/ul/li[3]/span[2]/span[@class="uppercase"]').text

		institution = driver.find_element_by_xpath('.//*[@id="course-summary-area"]/ul/li[4]/span[2]/a').text

		subject = driver.find_element_by_xpath('.//*[@id="course-summary-area"]/ul/li[5]/span[2]/a').text

		level = driver.find_element_by_xpath('.//*[@id="course-summary-area"]/ul/li[6]/span[2][@class="block-list__desc"]').text

		languages = driver.find_element_by_xpath('.//*[@id="course-summary-area"]/ul/li[7]/span[2]/span').text

		video_transcripts = driver.find_element_by_xpath('.//*[@id="course-summary-area"]/ul/li[8]/span[2]/span').text

		prerequisites = driver.find_element_by_xpath('.//*[@id="course-summary-area"]/div[2]/p').text


		course_dict['link'] = course_link
		course_dict['title'] = title
		course_dict['short_description'] = short_description
		course_dict['length'] = length
		course_dict['effort'] = effort
		course_dict['price'] = price
		course_dict['institution'] = institution
		course_dict['subject'] = subject
		course_dict['level'] = level
		course_dict['languages'] = languages
		course_dict['video_transcripts'] = video_transcripts
		course_dict['prerequisites'] = prerequisites
		writer.writerow(course_dict.values())
		driver.close()
        
	except Exception as e:
		print(e)
		print(course_link)
		pass
    
	finally:
        

Message: no such element: Unable to locate element: {"method":"xpath","selector":".//*[@id="course-summary-area"]/div[2]/p"}
  (Session info: chrome=59.0.3071.115)
  (Driver info: chromedriver=2.31.488774 (7e15618d1bf16df8bf0ecf2914ed1964a387ba0b),platform=Mac OS X 10.10.5 x86_64)

https://www.edx.org/course/introduction-data-analysis-using-excel-microsoft-dat205x-1
Message: no such element: Unable to locate element: {"method":"xpath","selector":".//*[@id="course-summary-area"]/ul/li[8]/span[2]/span"}
  (Session info: chrome=59.0.3071.115)
  (Driver info: chromedriver=2.31.488774 (7e15618d1bf16df8bf0ecf2914ed1964a387ba0b),platform=Mac OS X 10.10.5 x86_64)

https://www.edx.org/course/immune-system-new-developments-research-osakaux-bio101x-0
Message: no such element: Unable to locate element: {"method":"xpath","selector":".//*[@id="course-summary-area"]/ul/li[3]/span[2]/span[@class="uppercase"]"}
  (Session info: chrome=59.0.3071.115)
  (Driver info: chromedriver=2.31.488774 (7e15618d1bf

In [41]:
csv_file.close()

In [10]:
course_dict


{'effort': '3-5 hours per week',
 'institution': 'HKUSTx',
 'languages': 'English',
 'length': '5 weeks',
 'level': 'Introductory',
 'prerequisites': 'Introduction to Java Programming Part 1 - COMP102.1x is recommended. ',
 'price': 'FREE',
 'short_description': 'The first MOOC to teach the fundamental elements of Java programming and data abstraction.',
 'subject': 'Computer Science',
 'title': 'Introduction to Java Programming – Part 2',
 'video_transcripts': 'English'}

In [12]:
course_links.index('https://www.edx.org/course/introduction-biology-secret-life-mitx-7-00x-6')

479

In [14]:
course_links[200]

'https://www.edx.org/course/biobased-principles-opportunities-wageningenx-bb01x-0'

In [16]:
course_links.index('https://www.edx.org/course/digital-branding-engagement-curtinx-mkt1x-3')

244